# Annual Financial Statements - SimFin API

**Luiz Guilherme Gomes Fregona**

12/10/2021

## Table of contents
1. [Introduction](#introduction)
2. [Objetive](#objetive)
3. [Pipeline](#pipeline)
4. [Imports](#imports)
5. [Extract Data from API](#extraction)
    1. [Call the API](#api)
    2. [Save as Dataframe](#save)
7. [Extract, Transform, and Load (ETL)](#etl)
    1. [Set Paths](#paths)
    2. [Extract](#extract)
    3. [Tranform](#transform)
    4. [Load](#load)
    5. [Logging](#logging)
    6. [Running ETL process](#running)

## Introduction <a name="introduction"></a>

The use of data for prediction of stocks has grown in the last 5 years. Both robots and analysts have started to make predictions, and estimates based on financial data, which are regulated and can be found in SEC EDGAR website. In this project, we focused on acquiring financial data from an open-source API called SimFin. The main documents extracted from the API are the income statement, balance sheet, and cash flow of Apple and Google. 

## Objetives <a name="objetive"></a>

The overall goal of this project is to develop an ETL process capable of unifying all three documents into a single .csv file.

## Pipeline <a name="pipeline"></a>

The pipeline decided for this project includes 4 major steps:

1. Extract financial documents from API, and convert each one separetely .csv files
2. Extract .csv files from local repo and store them into a pandas dataframe
3. Tranform data following certain criterias
4. Load the unified dataframe into a final .csv file

## Imports <a name="imports"></a>

Some libraries will be required before moving forward with this project. The following code check if you already have those libraries.

In [4]:
#Check if the packages are installed in the desktop
try:
    import requests
except ImportError:
    sys.exit("You need the requests package! Run !pip install requests and try again.")
    
try:
    import pandas
except ImportError:
    sys.exit("You need the pandas package! Run !pip install pandas and try again.")

try:
    import os
except ImportError:
    sys.exit("You need the os package! Run !pip install os and try again.")
    
try:
    import numpy
except ImportError:
    sys.exit("You need the os package! Run !pip install numpy and try again.")

try:
    from datetime import datetime
except ImportError:
    sys.exit("You need the os package! Run !pip install datetime and try again.")

try:
    import glob
except ImportError:
    sys.exit("You need the os package! Run !pip install glob and try again.")

In case, you don't have them. Please remove the "#" symbol from the code cells below before running them.

In [9]:
#!pip install requests
#!pip install pandas
#!pip install os
#!pip install numpy
#!pip install datetime
#!pip install glob

## Extract Data from API <a name="extraction"></a>

SimFin is a open-source API which provides financial statements and fundamentals about the main public companies in US. SimFin provides two types of APIs: Python API and WEB API. Here, we are going to use the WEB API. The API key was obtained at https://simfin.com/data/api.

In [10]:
api_key = "f6Dh7PNT3yDf1DRct0XJ2tdeZhY2U8SQ"

As explained before, the goal is to get the income statement, the balance sheet, and the cash flow from apple & google since 2010 until now. 

In [11]:
#Define the stock ticker
tickers = ["AAPL","GOOG"]

#Define the periods and years
period = "fy"
year_start = 2010
year_end = 2020

In [12]:
#Endpoint for all financial statements
request_url = 'https://simfin.com/api/v2/companies/statements'

### Call the API <a name="api"></a>

Before calling the API, we are going to set all scientific notation to 5 decimal points

In [ ]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In order to retrieve matching filings from the API, we are going to send a HTTP POST request to the API with a dictionary formatted payload for each combinating of years, periods, and tickers. The query format follows the recomendations presented at https://simfin.com/api/v2/documentation/#tag/Company/paths/~1companies~1list/get.

In [13]:
#Income Statements

# variable to store the names of the columns
income_columns = []
# variable to store our data
income_data = []

for ticker in tickers:
    
    #loop through all years
    for year in range(year_start, year_end + 1):

                # define the parameters for the query
                parameters = {"statement": "pl", "ticker": ticker, "period": period, "fyear": year, "api-key": api_key}
                # make the request
                request = requests.get(request_url, parameters)

                # convert response to json and take 0th index
                data = request.json()[0]

                # make sure that data was found
                if data['found'] and len(data['data']) > 0:
                    # add the column descriptions once only
                    if len(income_columns) == 0:
                        income_columns = data['columns']
                    # add the data
                    income_data += data['data']

In [14]:
#Balance Sheet

# variable to store the names of the columns
balance_columns = []
# variable to store our data
balance_data = []

for ticker in tickers:
    
    #loop through all years
    for year in range(year_start, year_end + 1):

                # define the parameters for the query
                parameters = {"statement": "bs", "ticker": ticker, "period": period, "fyear": year, "api-key": api_key}
                # make the request
                request = requests.get(request_url, parameters)

                # convert response to json and take 0th index
                data = request.json()[0]

                # make sure that data was found
                if data['found'] and len(data['data']) > 0:
                    # add the column descriptions once only
                    if len(balance_columns) == 0:
                        balance_columns = data['columns']
                    # add the data
                    balance_data += data['data']

In [15]:
#Cash Flow

# variable to store the names of the columns
flow_columns = []
# variable to store our data
flow_data = []

for ticker in tickers:
    
    #loop through all years
    for year in range(year_start, year_end + 1):

                # define the parameters for the query
                parameters = {"statement": "cf", "ticker": ticker, "period": period, "fyear": year, "api-key": api_key}
                # make the request
                request = requests.get(request_url, parameters)

                # convert response to json and take 0th index
                data = request.json()[0]

                # make sure that data was found
                if data['found'] and len(data['data']) > 0:
                    # add the column descriptions once only
                    if len(flow_columns) == 0:
                        flow_columns = data['columns']
                    # add the data
                    flow_data += data['data']

### Save as Dataframe <a name="save"></a>

In order to keep data organized, the data will be stored in .csv files separately.

In [ ]:
#Build dataframes
df_income = pd.DataFrame(income_data, columns = income_columns)
df_balance = pd.DataFrame(balance_data, columns = balance_columns)
df_cash_flow = pd.DataFrame(flow_data, columns = flow_columns)

#Check what is the current working directory
os.getcwd()

#Save as .csv files
df_income.to_csv("income.csv", index = False)
df_balance.to_csv("balance.csv", index = False) 
df_cash_flow.to_csv("cash_flow.csv", index = False)

## Extract, Transform, and Load (ETL) <a name="etl"></a>

To keep the whole process of Extract, Tranform, and Load organized, a set of documents and functions will be created first. The documents are a .csv file, where all the information will be stored and read to be uploaded to a relational database, and a .txt logging file to keep track of all steps throughout the ETL process. 

### Set Paths <a name="paths"></a>

In [ ]:
logfile    = "logfile.txt"      # all event logs will be stored in this file
targetfile = "final_data.csv"   # file where transformed data is stored

### Extract <a name="extract"></a>

The extract function converts all .csv files into one single dataframe.

In [ ]:
#CSV extract function
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe

In [ ]:
#Files extract function
def extract():
    
    # variable to store the names of the columns
    columns = []
    # variable to store our data
    data = []
    
    #process all csv files
    for csvfile in glob.glob("*.csv"):
            raw_data = extract_from_csv(csvfile)
            extracted_columns = raw_data.columns
            extracted_list = raw_data.transpose().to_numpy().tolist()
            columns.extend(extracted_columns)
            data.extend(extracted_list)

    #Transpose data list
    data = np.array(data).T.tolist()

    #Build final dataframe
    extracted_data = pd.DataFrame(data, columns=columns)
    
    return extracted_data

### Transform <a name="transform"></a>

The transform function does the following tasks:

1. Fill "nan" with 0
2. Get rid of all unnecessary columns
3. Define all datatypes correctly

In [ ]:
def transform(data):
        #Fill "nan" values with 0
        data = data.replace("nan", 0)
        
        #Filter duplicated columns
        data = data.loc[:,~data.columns.duplicated()]
        
        #Get rid of useless columns
        useless_col = ["SimFinId","Fiscal Period", "Restated Date", "Publish Date", "Source", "TTM", "Value Check"]
        data = data[data.columns[~data.columns.isin(useless_col)]]
        
        #Convert categorical objects into strings, and date into date
        data['Report Date'] =  pd.to_datetime(data['Report Date'], utc = False)
        data[['Ticker','Fiscal Year']] = data[['Ticker','Fiscal Year']].astype(str)
        
        #The remaining objects were turn into numeric features
        col = data.columns.drop(['Ticker','Fiscal Year','Report Date'])
        data[col] = data[col].apply(pd.to_numeric)
        
        return data

### Load <a name="load"></a>

The load function add the final dataframe to a .csv file ready to be uploaded into a database.

In [ ]:
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile)  

### Logging <a name="logging"></a>

The logging function help us keep track of all updates performed during the ETL process.

In [ ]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

### Running ETL process  <a name="running"></a>

In [ ]:
log("ETL Job Started")

In [ ]:
log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")

In [ ]:
log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")

In [ ]:
log("Load phase Started")
load(targetfile,transformed_data)
log("Load phase Ended")

In [ ]:
log("ETL Job Ended")